# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [63]:
import sqlite3
import nltk

import random
import numpy as np
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.tokenize import sent_tokenize

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [64]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [34]:
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = convention_cur.fetchall()
print(tables)

[('conventions',)]


In [35]:
convention_cur.execute("PRAGMA table_info(conventions);")
columns = convention_cur.fetchall()
print(columns)

[(0, 'party', 'TEXT', 0, None, 0), (1, 'night', 'INTEGER', 0, None, 0), (2, 'speaker', 'TEXT', 0, None, 0), (3, 'speaker_count', 'INTEGER', 0, None, 0), (4, 'time', 'TEXT', 0, None, 0), (5, 'text', 'TEXT', 0, None, 0), (6, 'text_len', 'TEXT', 0, None, 0), (7, 'file', 'TEXT', 0, None, 0)]


In [72]:
convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions
                            WHERE party != 'Other';
                            ''')

for row in query_results :
    speech_text, party = row
    convention_data.append([speech_text, party])


In [57]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right. 

In [67]:
random.choices(convention_data,k=5)

[['How is your family going.', 'Democratic'],
 ['Great.', 'Republican'],
 ['For me, the most important issue is jobs. We need to be able to pay our bills. I trust Donald Trump on jobs way more than I do Joe Biden. There is no doubt that Donald Trump has built the strongest economy we’ve ever seen and I know that he will do it again. Joe Biden doesn’t have the energy and his ideas like raising taxes, it’s going to hurt us. Electing Joe Biden as the president is a huge risk and I don’t want the future of my son in the hands of Joe Biden. I want my son to be able to go out and get a good education. I want the economy to be strong. I want the country that he lives in to be safe and I want our borders to be secure and I don’t want that in Joe Biden’s hands. Donald Trump has built the strongest economy we’ve ever seen, he has made our country safe and as a mom, those are things that I’m very proud of and those are things that are very important to me when I’m looking for a president. I am st

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps: 

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [83]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions
                            WHERE party != 'Other';
                            ''')

for row in query_results :
    speech_text, party = row
    convention_data.append([speech_text, party])

def clean_tokenize(text):
    tokens = word_tokenize(text.lower())
    return ' '.join(cleaned_tokens)
    cleaned_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

for row in query_results:
    speech_text = row[0]  
    party = row[1]  
    
    cleaned_speech = clean_tokenize(speech_text)
    
    convention_data.append([cleaned_speech, party])

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [84]:
random.choices(convention_data,k=5)

[['This November, we stand at a crossroad. Mobs torch our cities while popular members of Congress promote the same socialism that my father fought against in World War II. We have a Democratic candidate for president who says that I’m not black if I don’t vote for him. Now more than ever, we need leaders who stand by their principles, they won’t compromise their values for political opportunities. Now more than ever, we need leaders who will stand up to the lawlessness supported by the radical left. This November, we have an opportunity to reject the mob mentality and once again be the America that my great great grandfather believed in. During the Trump administration, business ownership among blacks, Hispanics, and females have reached all time highs. Those same groups enjoyed record low unemployment and unprecedented prosperity, and we’re just getting started. I ran for Congress because we don’t need more career politicians, we need a few more chimney sweeps. We need more leaders l

In [85]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 0 as features in the model.


In [88]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    
def conv_features(text, fw):
    ret_dict = dict()
    tokens = text.split()
    for token in tokens:
        if token in fw:
            ret_dict[token] = True
    return ret_dict


In [90]:
assert(len(feature_words) > 0)
assert(conv_features("obama was the president", feature_words) ==
       {'obama': True, 'president': True})
assert(conv_features("some people in america are citizens", feature_words) ==
       {'people': True, 'america': True, 'citizens': True})

print("All assertions passed!")

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [91]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [92]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [93]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.616


In [96]:
accuracy = nltk.classify.accuracy(classifier, test_set)
print(f"Accuracy of the Naive Bayes Classifier: {accuracy * 100:.2f}%")

Accuracy of the Naive Bayes Classifier: 61.60%


Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

I had a lot of trouble with this code, due to errors with my machine running. With the point I was able to run it to, the Naive Bayes Classifier ran with an accuracy of 61.60%. The model accurately separates Democratic and Republicans based on text features nearly 62% of the time. This definitely would need some fine tuning before being deployed. 



In [102]:
convention_db.close()

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [133]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [134]:
results = cong_cur.execute('''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results)

In [137]:
tweet_data = []
for row in results:
    tweet_text = row[2]  # tweet_text
    party = row[1]  # party affiliation

In [142]:
def clean_tokenize(text):
    if isinstance(text, bytes):
        text = text.decode('utf-8')
    
    text = text.lower()  
    tokens = text.split()  
    
   
    cleaned_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    
    return ' '.join(cleaned_tokens)

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [149]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

### Reflections

_Write a little about what you see in the results_ 